In [21]:
import sympy as sym
from sympy import Sum

n, t, y, z= sym.symbols('n t y z')
x = sym.IndexedBase('x')
α = sym.IndexedBase('α')
k = sym.IndexedBase('k')
β = sym.IndexedBase('β')

i, j = sym.symbols('i j', cls=sym.Idx)

GLV = α[i]*(x[i])*(1-(x[i]/k[i])) + x[i] * (Sum(β[i,j]*x[j], (j , 1, n)))
GLV

(1 - x[i]/k[i])*x[i]*α[i] + x[i]*Sum(x[j]*β[i, j], (j, 1, n))

In [22]:
def get_glv_eq(n, i):
    return  α[i] * (x[i]) * (1 - (x[i] / k[i])) + x[i] * Sum(β[i,j] * x[j], (j , 1, n)).doit().subs(β[i,i], 0)

def solve_for(n, indexed_arr=None):

    x = sym.IndexedBase('x')
    eqs_arr = [get_glv_eq(n, i) for i in range(1, n + 1)]

    if indexed_arr == None:
        x_arr = [x[i] for i in range(1, n + 1)]
        sols = sym.solve(eqs_arr, x_arr)            #Calculo de equilibrios
    else:
        sols = sym.solve(eqs_arr, indexed_arr)      #Calculo de equilibrios  
          
    return sols

#### Solución para dos especies 
- n = 2

In [23]:
n = 2
eqs_arr = [get_glv_eq(n, i) for i in range(1, n + 1)]

In [24]:
eqs_arr[0]


(1 - x[1]/k[1])*x[1]*α[1] + x[1]*x[2]*β[1, 2]

In [25]:
eqs_arr[1]

(1 - x[2]/k[2])*x[2]*α[2] + x[1]*x[2]*β[2, 1]

### Hallando los puntos de equilibrio tenemos:

In [26]:
sols2 = solve_for(n=2)
sols2

[(0, 0),
 (0, k[2]),
 ((-k[2]*β[1, 2] - α[1])*k[1]*α[2]/(k[1]*k[2]*β[1, 2]*β[2, 1] - α[1]*α[2]),
  (-k[1]*k[2]*α[1]*β[2, 1] - k[2]*α[1]*α[2])/(k[1]*k[2]*β[1, 2]*β[2, 1] - α[1]*α[2])),
 (k[1], 0)]

In [27]:
# Cambiar el índice "indx" de para evaluar diferentes equilibrios.
indx = 2
x1_eq =sym.simplify(sols2[indx][0]) #SELECCIONAMOS, SOLS TIENE EN LA PRIMERA DIMENSION EL NUMERO DEL EQUILIBRIO, Y LA SEGUNDA ES PARA X Y PARA Y. 
x2_eq =sym.simplify(sols2[indx][1])
print(f"{x[1]} = {x1_eq}") # X
print(f"{x[2]} = {x2_eq}") # Y

x[1] = -(k[2]*β[1, 2] + α[1])*k[1]*α[2]/(k[1]*k[2]*β[1, 2]*β[2, 1] - α[1]*α[2])
x[2] = (-k[1]*β[2, 1] - α[2])*k[2]*α[1]/(k[1]*k[2]*β[1, 2]*β[2, 1] - α[1]*α[2])


In [28]:
x1_eq

-(k[2]*β[1, 2] + α[1])*k[1]*α[2]/(k[1]*k[2]*β[1, 2]*β[2, 1] - α[1]*α[2])

In [29]:
x2_eq

(-k[1]*β[2, 1] - α[2])*k[2]*α[1]/(k[1]*k[2]*β[1, 2]*β[2, 1] - α[1]*α[2])

### Hallando el Jacobiano tenemos:

In [30]:
Meq=sym.Matrix(eqs_arr) # matriz de ecuaciones
Mvar=sym.Matrix([x[1], x[2]]) # matriz de variables

Jac = Meq.jacobian(Mvar) # Jacobiano
Jac

Matrix([
[(1 - x[1]/k[1])*α[1] + x[2]*β[1, 2] - x[1]*α[1]/k[1],                                         x[1]*β[1, 2]],
[                                        x[2]*β[2, 1], (1 - x[2]/k[2])*α[2] + x[1]*β[2, 1] - x[2]*α[2]/k[2]]])

In [31]:
# Evaluando el Jacobiano en puntos de equilibrio:
Jaceq = Jac.subs(x[1],x1_eq).subs(x[2],x2_eq)
Jaceq

Matrix([
[(-k[1]*β[2, 1] - α[2])*k[2]*α[1]*β[1, 2]/(k[1]*k[2]*β[1, 2]*β[2, 1] - α[1]*α[2]) + (k[2]*β[1, 2] + α[1])*α[1]*α[2]/(k[1]*k[2]*β[1, 2]*β[2, 1] - α[1]*α[2]) + ((k[2]*β[1, 2] + α[1])*α[2]/(k[1]*k[2]*β[1, 2]*β[2, 1] - α[1]*α[2]) + 1)*α[1],                                                                                                                                                              -(k[2]*β[1, 2] + α[1])*k[1]*α[2]*β[1, 2]/(k[1]*k[2]*β[1, 2]*β[2, 1] - α[1]*α[2])],
[                                                                                                                                                          (-k[1]*β[2, 1] - α[2])*k[2]*α[1]*β[2, 1]/(k[1]*k[2]*β[1, 2]*β[2, 1] - α[1]*α[2]), -(-k[1]*β[2, 1] - α[2])*α[1]*α[2]/(k[1]*k[2]*β[1, 2]*β[2, 1] - α[1]*α[2]) - (k[2]*β[1, 2] + α[1])*k[1]*α[2]*β[2, 1]/(k[1]*k[2]*β[1, 2]*β[2, 1] - α[1]*α[2]) + (-(-k[1]*β[2, 1] - α[2])*α[1]/(k[1]*k[2]*β[1, 2]*β[2, 1] - α[1]*α[2]) + 1)*α[2]]])

#### Hallando valores propios tenemos:

In [32]:
##VALORES PROPIOS
eig = Jaceq.eigenvals()
Eig = list(eig.keys())
Eig1 = Eig[0]
Eig2 = Eig[1]

Valor propio 1:

In [33]:
sym.simplify(Eig1)

(-sqrt((4*(k[1]*k[2]*β[1, 2]*β[2, 1] - α[1]*α[2])*(k[1]*k[2]*β[1, 2]*β[2, 1] + k[1]*α[1]*β[2, 1] + k[2]*α[2]*β[1, 2] + α[1]*α[2]) + (k[1]*β[2, 1] + k[2]*β[1, 2] + α[1] + α[2])**2*α[1]*α[2])*α[1]*α[2]) + (k[1]*β[2, 1] + k[2]*β[1, 2] + α[1] + α[2])*α[1]*α[2])/(2*(k[1]*k[2]*β[1, 2]*β[2, 1] - α[1]*α[2]))

Valor propio 2:

In [34]:
sym.simplify(Eig2)

(sqrt((4*(k[1]*k[2]*β[1, 2]*β[2, 1] - α[1]*α[2])*(k[1]*k[2]*β[1, 2]*β[2, 1] + k[1]*α[1]*β[2, 1] + k[2]*α[2]*β[1, 2] + α[1]*α[2]) + (k[1]*β[2, 1] + k[2]*β[1, 2] + α[1] + α[2])**2*α[1]*α[2])*α[1]*α[2]) + (k[1]*β[2, 1] + k[2]*β[1, 2] + α[1] + α[2])*α[1]*α[2])/(2*(k[1]*k[2]*β[1, 2]*β[2, 1] - α[1]*α[2]))

#### Solución para tres especies 
- n = 3

In [35]:
n = 3
eqs_arr = [get_glv_eq(n, i) for i in range(1, n + 1)]

In [36]:
eqs_arr[0]

(1 - x[1]/k[1])*x[1]*α[1] + (x[2]*β[1, 2] + x[3]*β[1, 3])*x[1]

In [37]:
eqs_arr[1]

(1 - x[2]/k[2])*x[2]*α[2] + (x[1]*β[2, 1] + x[3]*β[2, 3])*x[2]

In [38]:
eqs_arr[2]

(1 - x[3]/k[3])*x[3]*α[3] + (x[1]*β[3, 1] + x[2]*β[3, 2])*x[3]

Tomando las expresiones para incluirlas en una simulación de 3 especies:

In [39]:
[print(eqs_arr[i].subs(x[1], x).subs(x[2], y).subs(x[3], z)) for i in range(n)];

(1 - x/k[1])*α[1]*x + (y*β[1, 2] + z*β[1, 3])*x
y*(-y/k[2] + 1)*α[2] + y*(z*β[2, 3] + β[2, 1]*x)
z*(y*β[3, 2] + β[3, 1]*x) + z*(-z/k[3] + 1)*α[3]


- Hay que tener cuidado con los β, pues son 4 o 6 en n = 2 o 3 especies respectivamente.
- Podríamos asumir que β[1, 2] = β[2, 1] y así para todos, pero creo que algo se puede hacer teniendo esos β individuales.

- Recordar que β[A, B] es como se ve afectado el crecimiento de la especie A debido a la especie B.
- Recordar que x[1], x[2], x[3] = x, y, z.
- Por ahí leí que si n >= 4 el sistema es caótico:
https://stefanoallesina.github.io/Sao_Paulo_School/intro.html#multi-species-dynamics
- https://www.frontiersin.org/articles/10.3389/fmicb.2019.00288/full

### 

### Hallando los puntos de equilibrio tenemos: 
#### Nota: Tomará varios minutos y no he obtenido un resultado tras 5 min y una RAM casi llena :'v

In [40]:
# sols3 = solve_for(n=3)
# sols3